In [10]:
import pandas as pd
import numpy as np
import json
import datetime
import random

In [11]:
with open("superedge_locations_dates.json", 'r') as f:
    dates = json.load(f)

In [12]:
def time_converter(time_stamp):
    normalizer = 1000;
    return datetime.datetime.fromtimestamp(time_stamp/normalizer).strftime('%Y')

In [13]:
new_dates = {k: time_converter(v) for k, v in dates['publicationDate'].items()}

The list of tuples show the whole range od years that our dataset span togeather with the total number of news in each year


In [14]:
count_dict = dict()
for i in new_dates:
    year = new_dates[i]
    if year in count_dict:
        count_dict[year] += 1
    else:
        count_dict[year] = 1
print(sorted(count_dict.items(),reverse=True))

[('2017', 135), ('2016', 331), ('2015', 492), ('2014', 461), ('2013', 336), ('2012', 331), ('2011', 444), ('2010', 504), ('2009', 635), ('2008', 634), ('2007', 577), ('2006', 607), ('2005', 490), ('2004', 462), ('2003', 547), ('2002', 932), ('2001', 710), ('2000', 838), ('1999', 950), ('1998', 873), ('1997', 726), ('1996', 423), ('1995', 544), ('1994', 533), ('1993', 475), ('1992', 613), ('1991', 181), ('1990', 232), ('1989', 237), ('1988', 223), ('1987', 265), ('1986', 299), ('1985', 200), ('1984', 218), ('1983', 187), ('1982', 140), ('1981', 127), ('1980', 142), ('1979', 103), ('1978', 94), ('1977', 78), ('1976', 108), ('1975', 149), ('1974', 132), ('1973', 201), ('1972', 194), ('1971', 135), ('1970', 142), ('1969', 97), ('1968', 108), ('1967', 240), ('1966', 399), ('1965', 349), ('1964', 310), ('1963', 250), ('1962', 167), ('1961', 157), ('1960', 151), ('1959', 191), ('1958', 180), ('1957', 126), ('1956', 206), ('1955', 191), ('1954', 418), ('1953', 302), ('1952', 207), ('1951', 219

In [15]:
with open("superedge_locations.json", 'r') as f:
    connections = json.load(f)

In [16]:
with open("newloc2coord.json", 'r') as f:
    standart_names = json.load(f)

In [17]:
canton_names = list()
for row in pd.read_json("ch.json")['features']:
    canton_names.append(row['properties']['name'])

Now we standardize the names of locations 

In [18]:
standartized_connections = dict()
for i  in connections:
    try:
        standartized_connections[i] = [standart_names[loc] for loc in connections[i]]
    except:
        pass

In [19]:
coonections_w_dates = dict()
for k,v in standartized_connections.items():
    connect_list = []
    for loc in v:
        if loc !=None and loc in canton_names and loc not in connect_list :
            connect_list.append(loc)
    coonections_w_dates[k] = (connect_list,new_dates[k])

In [20]:
cleaned_connections = dict()
for l in standartized_connections.values():
    for loc in l:
        if loc:
            cleaned_connections[loc] = dict()

for year in count_dict:
    for loc in cleaned_connections:
        cleaned_connections[loc][year] = list()

In [21]:
for k in coonections_w_dates:
    connections = coonections_w_dates[k][0]
    year = coonections_w_dates[k][1]
    if len(connections) > 2:
        for loc in connections:
            cleaned_connections[loc][year].append({'id':k,'news':connections})

In [22]:
resulting_connections = dict()
for loc in cleaned_connections:
    if loc in canton_names:
        resulting_connections[loc] = cleaned_connections[loc]

Now we don't standardize the names of locations 

In [23]:
with open("superedge_locations.json", 'r') as f:
    connections = json.load(f)

In [24]:
with open("newloc2coord.json", 'r') as f:
    standart_names = json.load(f)

In [25]:
raw_connections = dict()
for i in connections:
    try:
        raw_connections[i] = [standart_names[loc] for loc in connections[i]]
        raw_connections[i] = [loc for loc in connections[i]]
    except:
        pass

In [26]:
raw_connections_w_date = dict()
for k,v in raw_connections.items():
    connect_list = []
    for loc in v:
        if loc !=None and standart_names[loc] in canton_names and loc not in connect_list:
            connect_list.append(loc)
    raw_connections_w_date[k] = (connect_list,new_dates[k])

In [27]:
connection_loc_year = dict()
seen_connection = dict()
for l in standartized_connections.values():
    for loc in l:
        if loc:
            connection_loc_year[loc] = dict()
            seen_connection[loc] = dict()

for year in count_dict:
    for loc in connection_loc_year:
        connection_loc_year[loc][year] = list()
        seen_connection[loc][year] = list()

In [28]:
for k in raw_connections_w_date:
    connections = raw_connections_w_date[k][0]
    year = coonections_w_dates[k][1]
    if len(connections) > 2:
        for loc in connections:
            if connections not in seen_connection[standart_names[loc]][year]:
                seen_connection[standart_names[loc]][year].append(connections)
                connection_loc_year[standart_names[loc]][year].append({'id':k,'news':connections})

In [29]:
resulting_connections = dict()
for loc in connection_loc_year:
    if loc in canton_names:
        resulting_connections[loc] = connection_loc_year[loc]

In [32]:
# SAVE 
with open('canton_raw_connections.js', 'w') as outfile:
    json.dump(resulting_connections, outfile, ensure_ascii=False)

In [30]:
yearlyConnections = dict()
for year in count_dict:
    yearlyConnections[year] = set()
    
for canton in resulting_connections:
    for year in resulting_connections[canton]:
        n = 0
        for connection in resulting_connections[canton][year]:
            for loc in connection["news"]:
                # take one loc from each connection
                # if n > len(resulting_connections[canton][year]):
                #     break
                # else:
                #     n += 1
                yearlyConnections[year].add(loc)
                

for year in yearlyConnections:
    yearlyConnections[year] = list(yearlyConnections[year])

In [31]:
with open('yearly_connections.js', 'w') as outfile:
    json.dump(yearlyConnections, outfile, ensure_ascii=False)

### `Canton Locations`

In [22]:
canton_coordinates = dict()
for canton in pd.read_json("ch.json")['features']:
    df = pd.DataFrame(canton['geometry']['coordinates'][0][0])
    canton_coordinates[canton['properties']['name']] = (df.mean(axis=0)[1].round(5), df.mean(axis=0)[0].round(5))

In [24]:
canton_coordinates['Aargau']

(47.392890000000001, 8.1405700000000003)

In [22]:
with open('canton_coordinates.js', 'w') as outfile:
    json.dump(canton_coordinates, outfile, ensure_ascii=False)

### Excerpts

In [21]:
with open("excerpts_filtered.json", 'r') as f:
    exerpts = json.load(f)

In [22]:
len(standartized_connections)

23148

In [23]:
exerpts_filtered = {k:exerpts[k] for k in standartized_connections.keys()}

In [24]:
with open("excerpts_filtered.js", 'w') as f:
    json.dump(exerpts_filtered,f)

In [25]:
len(exerpts_filtered)

23148

In [26]:
exerpts_filtered

{'28': {'alternativeExcerpt': '',
  'excerpt': "Le sport, ce ne sont pas que des hommes, des femmes, des résultats ou des statistiques. Ce sont aussi des lieux. Certains ont marqué l'Histoire."},
 '30': {'alternativeExcerpt': '', 'excerpt': ''},
 '50': {'alternativeExcerpt': '',
  'excerpt': "Le sport, ce ne sont pas que des hommes, des femmes, des résultats ou des statistiques. Ce sont aussi des lieux. Certains ont marqué l'Histoire."},
 '89': {'alternativeExcerpt': '',
  'excerpt': "Maurice Tornay a brièvement évoqué à la radio Rhône FM  l'affaire  des problèmes fiscaux de Dominique Giroud, marchand de vin. Il s'est défendu de toute implication dans cette affaire de fraude fiscale.\r\n\r\n\r\n\r\n \r\n\r\n+ En direct par téléphone, Gaël Bourgeois, président du parti socialiste du Valais  romand."},
 '158': {'alternativeExcerpt': '',
  'excerpt': 'Reportage dans la classe de Noémie Chevalley, à l\'école de la Vignettaz, en ville de Fribourg, ainsi qu\'au CO de la Gruyère, à la Tour-de